This tutorial shows the potential of using CO2 capture resources in Macro such as Natural Gas Power CCS, SMR H2 CCS. Natural Gas Power CCS and SMR H2 CCS works by specifying a CO2 capture rate (for example 0.9) of the total CO2 emitted by natural gas used in producing power and H2 which effectively reduces the total emissions by 90%. There are a total of 2 CO2 node types: 1) Captured CO2 sink node, where its input is for captured CO2 from Natural Gas Power CCS and SMR H2 CCS, 2) Emission CO2 sink node to account for emission by fossil sources.

In [128]:
using Pkg
Pkg.activate(dirname(@__DIR__))

  Activating project at `~/Documents/GitHub/Macro`


In [129]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [130]:
T = 10*24;# 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [131]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
ng_fuel_price = df[1:T,:NG_Price]/NG_MWh; # $/MWh of natural gas
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne] # MWh of hydrogen

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

ngcc_inv_cost = 65400;# $/MW
ngcc_fom_cost = 10287.0;# $/MW
ngcc_vom_cost = 3.55; #$/MW
ngcc_capsize = 250.0;
ngcc_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas

ngcc_ccs_inv_cost = 66400;# $/MW
ngcc_ccs_fom_cost = 12287.0;# $/MW
ngcc_ccs_vom_cost = 3.65; #$/MW
ngcc_ccs_capsize = 250.0;
ngcc_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas
ngcc_ccs_CO2_captured_rate = 0.9; #Tonnes of CO2 captured/Tonness of CO2 produced by fuel

electrolyzer_capsize = 2*H2_MWh # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh) # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh # $/MW of H2
electrolyzer_fom_cost = 30500/H2_MWh # $/MW of H2
electrolyzer_vom_cost = 0.0;

smr_inv_cost = 1033333/H2_MWh;# $/MW of H2
smr_fom_cost = 20500/H2_MWh;# $/MW of H2
smr_vom_cost = 0.0; #$/MW of H2
smr_capsize = 20*H2_MWh;
smr_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas

smr_ccs_inv_cost = 1133333/H2_MWh;# $/MW of H2
smr_ccs_fom_cost = 21500/H2_MWh;# $/MW of H2
smr_ccs_vom_cost = 0.0; #$/MW of H2
smr_ccs_capsize = 20*H2_MWh;
smr_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tonnes of CO2 / MWh of natural gas
smr_ccs_CO2_captured_rate = 0.9; #Tonnes of CO2 captured/Tons of CO2 produced by fuel


In [132]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [133]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [134]:
solar_pv = Transformation{Electricity}(;
    id = :SolarPV,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    #### Note that this transformation does not have a stoichiometry balance because the sunshine is exogenous :-)
)
solar_pv.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = solar_pv,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
existing_capacity = 0.0,
capacity_factor = solar_capacity_factor,
investment_cost = solar_inv_cost,
fixed_om_cost = solar_fom_cost,
constraints = [Macro.CapacityConstraint()]
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electricity}(:SolarPV, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:E => TEdge{Electricity}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :output, true, false, true, 1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.5176, 0.3503, 0.1105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240,

We can also assigne a battery storage to the node:

In [135]:
battery = Transformation{Storage}(;
id = :battery,
stoichiometry_balance_names = [:storage],
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
can_expand = true,
can_retire = false,
existing_capacity_storage = 0.0,
investment_cost_storage = battery_inv_cost_storage,
fixed_om_cost_storage = battery_fom_cost_storage,
min_duration = 1,
max_duration = 10,
constraints = [Macro.StorageCapacityConstraint(),Macro.StoichiometryBalanceConstraint()],
discharge_capacity_edge = :discharge
)

battery.TEdges[:discharge] = TEdge{Electricity}(;
id = :discharge,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
node = e_node,
transformation = battery,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
existing_capacity = 0.0,
investment_cost = battery_inv_cost,
fixed_om_cost = battery_fom_cost,
variable_om_cost = battery_vom_cost,
st_coeff = Dict(:storage=>1/battery_eff_down),
constraints = [Macro.CapacityConstraint()]
)

battery.TEdges[:charge] = TEdge{Electricity}(;
id = :charge,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
node = e_node,
transformation = battery,
direction = :input,
has_planning_variables = false,
st_coeff = Dict(:storage=>battery_eff_up)
)


TEdge{Electricity}(:charge, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Storage}(:battery, 1:1:240, StepRange{Int64, Int64}[1:1:240], [:storage], Dict{Symbol, TEdge}(:charge => TEdge{Electricity}(#= circular reference @-3 =#), :discharge => TEdge{Electricity}(:discharge, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], D

We also consider a hydrogen demand node:

In [136]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  820.5846, 871.2461999999999, 846.9153, 796.2537, 655.9344, 481.6184999999999, 348.6318, 262.3071, 155.65109999999999, 87.9912], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We model an exogenous inflow of natural gas into the system using by introducing a natural gas source node (with zero demand):

In [137]:
ng_node = Node{NaturalGas}(;
    id = Symbol("NG_node"),
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    demand = zeros(length(time_interval(NaturalGas))),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint()]
)

Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

In [138]:
NGImport = Transformation{NaturalGas}(;
id = :NGImport,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
#### Note that this transformation does not have a stoichiometry balance because we are modeling exogenous inflow of NG
)

NGImport.TEdges[:ng_source] = TEdge{NaturalGas}(;
id = :ng_source,
node = ng_node,
transformation = NGImport,
direction = :output,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
has_planning_variables = false,
price = ng_fuel_price,
)

TEdge{NaturalGas}(:ng_source, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGas}(:NGImport, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:ng_source => TEdge{NaturalGas}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :output, false, false, false, 1.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240], Dict{Symbol, Float64}(), 0.0, Inf, 0.0, 0.0, 0.0, 0.0, [18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264,

We define an atmospheric CO2 node (which is also going to provide input int DAC with CO2 from the atmosphere to capture):

In [139]:
co2_node = Node{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0]
)


Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[])

We define captured CO2 node and sink to account for captured CO2. Instead of a sink, we could also define a storage resource!

In [140]:
co2_captured_node = Node{CO2Captured}(;
id = Symbol("CO2_captured_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint()]
)

co2_captured_sink = Transformation{CO2Captured}(;
id = :co2_captured_sink,
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
#### Note that this transformation does not have a stoichiometry balance because we are modeling just a sink
)

co2_captured_sink.TEdges[:co2_captured_flow] = TEdge{CO2Captured}(;
id = :co2_sink_flow,
node = co2_captured_node,
transformation = co2_captured_sink,
direction = :input,
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
has_planning_variables = false,
)

TEdge{CO2Captured}(:co2_sink_flow, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{CO2Captured}(:co2_captured_sink, 1:1:240, StepRange{Int64, Int64}[1:1:240], Symbol[], Dict{Symbol, TEdge}(:co2_captured_flow => TEdge{CO2Captured}(#= circular reference @-3 =#)), Dict{Any, Any}(), Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[], 0.0, Inf, 0.0, false, false, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, :.), :input, false, false, false, 1.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240], Dict{Symbol, Float64}(), 0.0, Inf, 0.0, 0.0, 0.0, 0.0, Float64[], 0.0, false, 1.0, 1.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeC

We are now ready to define a natural gas plant! Note that we require two stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas = CO2

This transformation has 3 edges: each edge carries a specific commodity flow

In [141]:
ngcc = Transformation{NaturalGasPower}(;
id = :NGCC,
time_interval = time_interval(Electricity),
stoichiometry_balance_names = [:energy,:emissions],
constraints = [Macro.StoichiometryBalanceConstraint()]
)

ngcc.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = Dict(:energy=>ngcc_heatrate,:emissions=>0.0),
existing_capacity = 0.0,
investment_cost = ngcc_inv_cost,
fixed_om_cost = ngcc_fom_cost,
variable_om_cost = ngcc_vom_cost,
constraints = [Macro.CapacityConstraint()]
)

ngcc.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>ngcc_fuel_CO2)
)

ngcc.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0)
)

TEdge{CO2}(:CO2, Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[]), Transformation{NaturalGasPower}(:NGCC, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions], Dict{Symbol, TEdge}(:NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(#= circular reference @-3 =#), :input, false, false, false, 1.0, Float64[], 1:1:240, StepRange{Int64, Int64}[1:1:240], Dict(:emis

We are now ready to define a natural gas plant with CCS! Note that we require three stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas $\times$ (1-CCS rate) = CO2

Fuel_CO2 $\times$ NaturalGas $\times$ CCS rate = CO2_Captured

This transformation has 4 edges: each edge carries a specific commodity flow

In [142]:
ngcc_ccs = Transformation{NaturalGasPowerCCS}(;
id = :NGCC_CCS,
time_interval = time_interval(Electricity),
stoichiometry_balance_names = [:energy,:emissions,:captured_emissions],
constraints = [Macro.StoichiometryBalanceConstraint()]
)

ngcc_ccs.TEdges[:E] = TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc_ccs,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_ccs_capsize,
time_interval = time_interval(Electricity),
subperiods = subperiods(Electricity),
st_coeff = Dict(:energy=>ngcc_ccs_heatrate,:emissions=>0.0,:captured_emissions=>0.0),
existing_capacity = 0.0,
investment_cost = ngcc_ccs_inv_cost,
fixed_om_cost = ngcc_ccs_fom_cost,
variable_om_cost = ngcc_ccs_vom_cost,
constraints = [Macro.CapacityConstraint()]
)

ngcc_ccs.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>ngcc_ccs_fuel_CO2*(1-ngcc_ccs_CO2_captured_rate),:captured_emissions=>ngcc_ccs_fuel_CO2*ngcc_ccs_CO2_captured_rate)
)

ngcc_ccs.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0,:captured_emissions=>0.0)
)

ngcc_ccs.TEdges[:CO2_Captured]=TEdge{CO2Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>0.0,:captured_emissions=>1.0)
)

TEdge{CO2Captured}(:CO2_Captured, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGasPowerCCS}(:NGCC_CCS, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions, :captured_emissions], Dict{Symbol, TEdge}(:CO2_Captured => TEdge{CO2Captured}(#= circular reference @-3 =#), :NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missin

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [143]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                stoichiometry_balance_names = [:energy],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

electrolyzer.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>1.0),
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

electrolyzer.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>electrolyzer_efficiency)
)

TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10262.0, 10332.0, 11008.0, 11778.0, 11691.0, 11401.0, 10973.0, 10250.0, 9297.0, 8468.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:240, StepRange{Int64, Int64}[], [:energy], Dict{Symbol, TEdge}(:H2 => TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  820.5846, 871.2461999999999, 846.9153, 796.2537, 655.9344, 481.6184999999999, 348.6318, 262.3071, 155.65109999999999, 87.9912], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, A

In [191]:
# electrolyzer2_data = Dict{Symbol, Any}(
#     :cap_size => 2 * H2_MWh,
#     :existing_cap => 0.0,
#     :efficiency => 1 / (45 / H2_MWh), 
#     :inv_cost => 2033333 / H2_MWh,
#     :fom_cost => 30500 / H2_MWh,
#     :vom_cost => 0.0,
#     :nodes => Dict{Symbol, Node}(
#         :h2_node => h2_node,
#         :elec_node => e_node
#     ),
#     :time_intervals => Dict{Symbol, StepRange{Int,Int}}(
#         :transform => time_interval(Electricity),
#         :H2 => time_interval(Hydrogen),
#         :E => time_interval(Electricity)
#     ),
#     :subperiods => Dict{Symbol, Vector{StepRange{Int,Int}}}(
#         :H2 => subperiods(Hydrogen),
#         :E => subperiods(Electricity)
#     )    
# )
# electrolyzer2 = Macro.make_electrolyzer(:electrolyzer2, electrolyzer2_data, electrolyzer2_data[:nodes][:h2_node], electrolyzer2_data[:nodes][:elec_node]);

We are now ready to define a natural gas hydrogen plant or SMR similar to a natural gas power plant

In [192]:
smr = Transformation{NaturalGasHydrogen}(;
                id = :SMR,
                time_interval = time_interval(Hydrogen),
                stoichiometry_balance_names = [:energy,:emissions],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

smr.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>smr_heatrate,:emissions=>0.0),
    existing_capacity = 0.0,
    investment_cost = smr_inv_cost,
    fixed_om_cost = smr_fom_cost,
    variable_om_cost = smr_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

smr.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>smr_fuel_CO2)
)

smr.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0)
)

TEdge{CO2}(:CO2, Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(Macro.CO2CapConstraint => 100000.0), Macro.AbstractTypeConstraint[Macro.CO2CapConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogen}(:SMR, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions], Dict{Symbol, TEdge}(:NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogen}(#= circular reference @-3 =#), :input, fa

Now, let's add a SMR hydrogen plant with CCS, similar to a natural gas power plant with CCS

In [193]:
smr_ccs = Transformation{NaturalGasHydrogenCCS}(;
                id = :SMR_CCS,
                time_interval = time_interval(Hydrogen),
                stoichiometry_balance_names = [:energy,:emissions,:captured_emissions],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

smr_ccs.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_ccs_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>smr_ccs_heatrate,:emissions=>0.0,:captured_emissions=>0.0),
    existing_capacity = 0.0,
    investment_cost = smr_ccs_inv_cost,
    fixed_om_cost = smr_ccs_fom_cost,
    variable_om_cost = smr_ccs_vom_cost,
    constraints = [Macro.CapacityConstraint()]
)

smr_ccs.TEdges[:NG] = TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr_ccs,
direction = :input,
has_planning_variables = false,
time_interval = time_interval(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = Dict(:energy=>1.0,:emissions=>smr_ccs_fuel_CO2*(1-smr_ccs_CO2_captured_rate),:captured_emissions=>smr_ccs_fuel_CO2*smr_ccs_CO2_captured_rate)
)

smr_ccs.TEdges[:CO2] = TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>1.0,:captured_emissions=>0.0)
)

smr_ccs.TEdges[:CO2_Captured] = TEdge{CO2Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:emissions=>0.0,:captured_emissions=>1.0)
)

TEdge{CO2Captured}(:CO2_Captured, Node{CO2Captured}(:CO2_captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogenCCS}(:SMR_CCS, 1:1:240, StepRange{Int64, Int64}[], [:energy, :emissions, :captured_emissions], Dict{Symbol, TEdge}(:CO2_Captured => TEdge{CO2Captured}(#= circular reference @-3 =#), :NG => TEdge{NaturalGas}(:NG, Node{NaturalGas}(:NG_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:240, StepRange{Int64, Int64}[1:1:240], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Dict{DataType, Float64}(), Dict{DataType, Float64}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(miss

Now that we have implemented different technologies, we can create the JuMP model.

In [194]:
nodes = [e_node; h2_node; ng_node; co2_node; co2_captured_node];
system = [nodes;solar_pv; battery; ngcc; ngcc_ccs; electrolyzer; electrolyzer2; smr; smr_ccs;NGImport;co2_captured_sink];

UndefVarError: UndefVarError: `electrolyzer2` not defined

In [195]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1)
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [196]:
Macro.add_planning_variables!.(system,model);

UndefVarError: UndefVarError: `system` not defined

In [197]:
Macro.add_operation_variables!.(system, model);

UndefVarError: UndefVarError: `system` not defined

In [198]:
Macro.add_all_model_constraints!.(system, model);

UndefVarError: UndefVarError: `system` not defined

In [200]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [201]:
using Gurobi

In [202]:
Macro.set_optimizer(model,Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-19


In [203]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 1 columns and 0 nonzeros
Model fingerprint: 0xa4f2e054
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00

User-callback calls 23, time in user-callback 0.00 sec


The installed electrolyzer capacity is:

In [204]:
Macro.value(Macro.capacity(electrolyzer.TEdges[:H2]))

KeyError: KeyError: key :capacity not found

The installed SMR capacity is

In [205]:
Macro.value(Macro.capacity(smr.TEdges[:H2]))

KeyError: KeyError: key :capacity not found

The installed SMR CCS capacity is

In [206]:
Macro.value(Macro.capacity(smr_ccs.TEdges[:H2]))

KeyError: KeyError: key :capacity not found

The installed solar capacity in MW is:

In [207]:
Macro.value(Macro.capacity(solar_pv.TEdges[:E]))

KeyError: KeyError: key :capacity not found

The installed battery capacity in MW is:

In [208]:
Macro.value(Macro.capacity(battery.TEdges[:discharge]))

KeyError: KeyError: key :capacity not found

The installed NGCC capacity in MW is:

In [209]:
Macro.value(Macro.capacity(ngcc.TEdges[:E]))

KeyError: KeyError: key :capacity not found

The installed NGCC CCS capacity in MW is:

In [210]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[:E]))

KeyError: KeyError: key :capacity not found

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [211]:
base_emissions  = sum(Macro.value.(Macro.flow(ngcc.TEdges[:CO2]))) +  sum(Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr_ccs.TEdges[:CO2])))

KeyError: KeyError: key :flow not found

The resulting captured CO2 (Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [212]:
base_CO2_captured  =  sum(Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2_Captured]))) + sum(Macro.value.(Macro.flow(smr_ccs.TEdges[:CO2_Captured])))

KeyError: KeyError: key :flow not found

Lets add a low CO2 cap:

In [213]:
merge!(co2_node.rhs_policy,Dict(Macro.CO2CapConstraint=>1e5))
append!(co2_node.constraints,[Macro.CO2CapConstraint()])

2-element Vector{Macro.AbstractTypeConstraint}:
 Macro.CO2CapConstraint(missing, missing, missing)
 Macro.CO2CapConstraint(missing, missing, missing)

In [214]:
Macro.add_model_constraint!(co2_node.constraints[1],co2_node,model)

KeyError: KeyError: key :net_balance not found

In [215]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 1 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00

User-callback calls 15, time in user-callback 0.00 sec


Now, the electrolyzer capacity is:

In [216]:
Macro.value(Macro.capacity(electrolyzer.TEdges[:H2]))

KeyError: KeyError: key :capacity not found

The SMR capacity is:

In [217]:
Macro.value(Macro.capacity(smr.TEdges[:H2]))

KeyError: KeyError: key :capacity not found

The SMR CCS capacity is:

In [218]:
Macro.value(Macro.capacity(smr_ccs.TEdges[:H2]))

KeyError: KeyError: key :capacity not found

Now the solar capacity is:

In [219]:
Macro.value(Macro.capacity(solar_pv.TEdges[:E]))

KeyError: KeyError: key :capacity not found

The installed battery capacity is:

In [220]:
Macro.value(Macro.capacity(battery.TEdges[:discharge]))

KeyError: KeyError: key :capacity not found

The installed NG capacity is:

In [221]:
Macro.value(Macro.capacity(ngcc.TEdges[:E]))

KeyError: KeyError: key :capacity not found

The installed NG CCS capacity is:

In [222]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[:E]))

KeyError: KeyError: key :capacity not found

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [223]:
sum(Macro.value.(Macro.flow(ngcc.TEdges[:CO2]))) +  sum(Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr_ccs.TEdges[:CO2])))

KeyError: KeyError: key :flow not found

The resulting captured CO2 (Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [224]:
sum(Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2_Captured]))) + sum(Macro.value.(Macro.flow(smr_ccs.TEdges[:CO2_Captured])))

KeyError: KeyError: key :flow not found

We see that in a low CO2 emission case, Power and H2 CCS are deployed along with PV and electrolyzers

However, if we use a net-zero CO2 cap:

In [225]:
co2_node.constraints[1].constraint_ref[1]

MethodError: MethodError: no method matching getindex(::Missing, ::Int64)

In [226]:
Macro.set_normalized_rhs(co2_node.constraints[1].constraint_ref[1], 0.0);

MethodError: MethodError: no method matching getindex(::Missing, ::Int64)

In [227]:
co2_node.constraints[1].constraint_ref[1]

MethodError: MethodError: no method matching getindex(::Missing, ::Int64)

In [228]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 1 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00

User-callback calls 15, time in user-callback 0.00 sec


Now, the electrolyzer capacity is:

In [229]:
Macro.value(Macro.capacity(electrolyzer.TEdges[:H2]))

KeyError: KeyError: key :capacity not found

The SMR capacity is:

In [230]:
Macro.value(Macro.capacity(smr.TEdges[:H2]))

KeyError: KeyError: key :capacity not found

The SMR CCS capacity is:

In [231]:
Macro.value(Macro.capacity(smr_ccs.TEdges[:H2]))

KeyError: KeyError: key :capacity not found

Now the solar capacity is:

In [232]:
Macro.value(Macro.capacity(solar_pv.TEdges[:E]))

KeyError: KeyError: key :capacity not found

The installed battery capacity is:

In [233]:
Macro.value(Macro.capacity(battery.TEdges[:discharge]))

KeyError: KeyError: key :capacity not found

The installed NG capacity is:

In [234]:
Macro.value(Macro.capacity(ngcc.TEdges[:E]))

KeyError: KeyError: key :capacity not found

The installed NG CCS capacity is:

In [235]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[:E]))

KeyError: KeyError: key :capacity not found

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [236]:
sum(Macro.value.(Macro.flow(ngcc.TEdges[:CO2]))) +  sum(Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr.TEdges[:CO2]))) + sum(Macro.value.(Macro.flow(smr_ccs.TEdges[:CO2])))

KeyError: KeyError: key :flow not found

The resulting captured CO2 (Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [237]:
sum(Macro.value.(Macro.flow(ngcc_ccs.TEdges[:CO2_Captured]))) + sum(Macro.value.(Macro.flow(smr_ccs.TEdges[:CO2_Captured])))

KeyError: KeyError: key :flow not found

We see that in a net-zero emission case, only PV power and electrolyzer H2 are deployed.